In [ ]:
!pip install bertopic

In [ ]:
import pandas as pd
from bertopic import BERTopic
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data = pd.read_csv("/content/YouTube.csv")
data

,ID,Video Title,Views,Likes,Dislikes,Transcript,class
0,hxwpkM5w3Cc,I Got Hunted By The FBI,42724724,2005151,24646,- I tied up an FBI agent.\n(words pop)If he st...,1
1,81mXAkFEkMs,10 Real Life Giants You Won't Believe Exist,6890718,15222,2858,hey what's happening y'all welcome backto anot...,1
2,9NQQKNAwaPk,10 Mythical CREATURES That Actually Existed,2089601,46750,1954,I used to taste your daddy who doesn'tlove a g...,1
3,Y_PQrPRolvM,Real Giants Caught On Camera!,518170,4033,271,Hi!\nWho said that giants are imaginary and on...,1
4,ZZtYAOpHVYk,10 Real Life GIANTS,9244943,64025,6508,[Music]hey guys what's up it's charlie here an...,1
...,...,...,...,...,...,...,...
150,x_qSdnjhbw4,Building an UNBREAKABLE TESSERACT CUBE!,1687124,81479,1474,"Of course, the main priority\nis to make the b...",0
151,PugcxQYJGjs,Lanky Kong listens to DK Rap for the first time,839745,102699,567,"- Yo! What up, guys?Lanky Kong here with\nanot...",0
152,cigmdYsgg_4,Can We Become American Ninja Warriors (Daequan...,819705,68459,402,all rightwhat's up guys welcome to the doom an...,0
153,s7MTM4BKZ_E,Why Anecdotes Trump Data,980055,36060,1347,Some critics of the TV show Mythbusters claimt...,0


In [ ]:
data["class"].value_counts()

0    84
1    71
Name: class, dtype: int64

In [ ]:
def clean_and_preprocess(text):
    # 1. Convert text to lowercase
    text = text.lower()

    # 2. Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 3. Tokenize the text into words
    tokens = word_tokenize(text)

    # 4. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatize words (convert words to their base form)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 6. Join the preprocessed tokens back into a clean text
    cleaned_text = ' '.join(tokens)

    return cleaned_text

In [ ]:
transcript = list(data["Transcript"])
video_title = list(data["Video Title"])

In [ ]:
cleaned_transcript_ = []
cleaned_video_title_ = []

similarity_score = []
for i in range(len(data)):
    cleaned_transcript = clean_and_preprocess(transcript[i])
    cleaned_video_title = clean_and_preprocess(video_title[i])

    cleaned_transcript_.append(cleaned_transcript)
    cleaned_video_title_.append(cleaned_video_title)

In [ ]:
# Load BertTokenizer and BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Tokenize and encode the video titles using BERT
encoded_titles = [tokenizer(title, padding=True, truncation=True, return_tensors='pt') for title in cleaned_video_title_]

# Extract embeddings from BERT for titles
title_embeddings = []
for encoded_title in encoded_titles:
    with torch.no_grad():
        outputs = model(**encoded_title)
    title_embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

In [ ]:
# Stack the title embeddings into a numpy array
title_embeddings_matrix = np.stack(title_embeddings)

In [ ]:
# Tokenize and encode the video transcripts using BERT
encoded_transcripts = [tokenizer(transcript, padding=True, truncation=True, return_tensors='pt') for transcript in cleaned_transcript_]

# Extract embeddings from BERT for transcripts
transcript_embeddings = []
for encoded_transcript in encoded_transcripts:
    with torch.no_grad():
        outputs = model(**encoded_transcript)
    transcript_embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

# Stack the transcript embeddings into a numpy array
transcript_embeddings_matrix = np.stack(transcript_embeddings)

In [ ]:
# Perform topic modeling using BERTopic for titles
title_bertopic = BERTopic(min_topic_size=2)  # Increase the min_topic_size if needed
title_topics, _ = title_bertopic.fit_transform(cleaned_video_title_)

# Perform topic modeling using BERTopic for transcripts
transcript_bertopic = BERTopic(min_topic_size=2)  # Increase the min_topic_size if needed
transcript_topics, _ = transcript_bertopic.fit_transform(cleaned_transcript_)

# Create DataFrames to display the results
title_results = pd.DataFrame({'Video Title': cleaned_video_title_, 'Topic (Title)': title_topics})
transcript_results = pd.DataFrame({'Transcript': cleaned_transcript_, 'Topic (Transcript)': transcript_topics})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
title_results

,Video Title,Topic (Title)
0,got hunted fbi,13
1,real life giant wont believe exist,0
2,mythical creature actually existed,0
3,real giant caught camera,0
4,real life giant,0
...,...,...
150,building unbreakable tesseract cube,-1
151,lanky kong listens dk rap first time,1
152,become american ninja warrior daequan hamlinz,-1
153,anecdote trump data,0


In [ ]:
transcript_results

,Transcript,Topic (Transcript)
0,tied fbi agent word popif steal hundred thousa...,2
1,hey whats happening yall welcome backto anothe...,21
2,used taste daddy doesntlove good story especia...,14
3,hi said giant imaginary onlyexist fairy tale m...,21
4,musichey guy whats charlie andtoday going look...,21
...,...,...
150,course main priority make box unbreakablebut a...,0
151,yo guyslanky kong another reaction videowe got...,-1
152,rightwhats guy welcome doom andninja warrior c...,-1
153,critic tv show mythbusters claimthat show misr...,8


In [ ]:
label_list = []
for i in range(len(data)):
  if title_results["Topic (Title)"][i] == transcript_results["Topic (Transcript)"][i]:
    label_list.append(0)
  else:
    label_list.append(1)

In [ ]:
true_labels = data["class"]

In [ ]:
True_clickbait = 0
False_clickbait = 0

True_nonclickbait = 0
False_nonclickbiat = 0

for i in range(len(true_labels)):
  if label_list[i] == 1 and  true_labels[i] == 1:
    True_clickbait += 1
  elif label_list[i] == 1 and  true_labels[i] == 0:
    False_clickbait += 1
  elif label_list[i] == 0 and  true_labels[i] == 1:
    False_nonclickbiat += 1
  elif label_list[i] == 0 and  true_labels[i] == 0:
    True_nonclickbait += 1

In [ ]:
print("True_clickbait", True_clickbait)
print("False_clickbait", False_clickbait)

print("True_nonclickbait", True_nonclickbait)
print("False_nonclickbiat", False_nonclickbiat)

True_clickbait 67
False_clickbait 78
True_nonclickbait 6
False_nonclickbiat 4


In [ ]:
accuracy = (True_clickbait + True_nonclickbait) / (True_clickbait + False_clickbait + True_nonclickbait + False_nonclickbiat)
accuracy

0.47096774193548385

In [ ]:
Precision = True_clickbait / (True_clickbait + False_clickbait)
Precision

0.46206896551724136

In [ ]:
Recall = True_clickbait / (True_clickbait + False_nonclickbiat)
Recall

0.9436619718309859

In [ ]:
scores_df = data
scores_df["Bert Feature"] = 0

for i in range (len(transcript_results)):
  if transcript_results["Topic (Transcript)"].iloc[i] == title_results["Topic (Title)"].iloc[i]:
    scores_df["Bert Feature"].iloc[i] = 0
  else:
    scores_df["Bert Feature"].iloc[i] = 1

scores_df = scores_df.drop(columns=["Video Title", "Transcript"])

scores_df


,ID,Views,Likes,Dislikes,class,Bert Feature
0,hxwpkM5w3Cc,42724724,2005151,24646,1,1
1,81mXAkFEkMs,6890718,15222,2858,1,1
2,9NQQKNAwaPk,2089601,46750,1954,1,1
3,Y_PQrPRolvM,518170,4033,271,1,1
4,ZZtYAOpHVYk,9244943,64025,6508,1,1
...,...,...,...,...,...,...
150,x_qSdnjhbw4,1687124,81479,1474,0,1
151,PugcxQYJGjs,839745,102699,567,0,1
152,cigmdYsgg_4,819705,68459,402,0,0
153,s7MTM4BKZ_E,980055,36060,1347,0,1


In [ ]:
scores_df["Bert Feature"].value_counts()

1    145
0     10
Name: Bert Feature, dtype: int64

In [ ]:
#SVM Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics

#Logistic Regression Imports
from sklearn import linear_model

#KNN Imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

def train_svm(df, df_xcols, df_ycol, kernel_type):
    x_svm = df[df_xcols]
    y_svm = df[df_ycol]
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(x_svm, y_svm, test_size=0.3) # 70% training and 30% test

    clf = SVC(kernel=kernel_type)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    #Model Accuracy
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

    #Model Precision
    print("Precision:",metrics.precision_score(y_test, y_pred))

    #Model Recall
    print("Recall:",metrics.recall_score(y_test, y_pred))

    #Model F1 Score
    print("F1 Score: ", metrics.f1_score(y_test, y_pred))

    return clf

def train_logr(df, feature_cols, df_ycol):
    x_logr = df[feature_cols]
    y_logr = df[df_ycol]
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(x_logr, y_logr, test_size=0.3)

    logr = linear_model.LogisticRegression(max_iter = 1000)
    logr.fit(X_train, y_train.values.ravel())
    y_pred = logr.predict(X_test)

    #Model Accuracy
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

    #Model Precision
    print("Precision:",metrics.precision_score(y_test, y_pred))

    #Model Recall
    print("Recall:",metrics.recall_score(y_test, y_pred))

    #Model F1 Score
    print("F1 Score: ", metrics.f1_score(y_test, y_pred))

    return logr

def train_KNN(df, feature_cols, df_ycol, neighbors):

    x_KNN = df[feature_cols]
    y_KNN = df[df_ycol]

    # Split into training and test set
    X_train, X_test, y_train, y_test = train_test_split(x_KNN, y_KNN, test_size = 0.3)

    knn = KNeighborsClassifier(n_neighbors=neighbors)

    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)

    #Model Accuracy
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

    #Model Precision
    print("Precision:",metrics.precision_score(y_test, y_pred))

    #Model Recall
    print("Recall:",metrics.recall_score(y_test, y_pred))

    #Model F1 Score
    print("F1 Score: ", metrics.f1_score(y_test, y_pred))

    return knn

In [ ]:
model = train_svm(scores_df, ['Views', 'Likes', 'Dislikes', 'Bert Feature'], 'class', 'rbf')

Accuracy: 0.723404255319149
Precision: 0.75
Recall: 0.47368421052631576
F1 Score:  0.5806451612903226


In [ ]:
model = train_logr(scores_df, ['Views', 'Likes', 'Dislikes', 'Bert Feature'], 'class')

Accuracy: 0.6808510638297872
Precision: 0.59375
Recall: 0.9047619047619048
F1 Score:  0.7169811320754718


In [ ]:
model = train_KNN(scores_df, ['Views', 'Likes', 'Dislikes', 'Bert Feature'], 'class', 7)

Accuracy: 0.8297872340425532
Precision: 0.8695652173913043
Recall: 0.8
F1 Score:  0.8333333333333333
